In [6]:

with open("SimpleStorage.sol","r") as file:
    simple_storage_file = file.read()
    #print(simple_storage_file)

# Steps
pip install web3
pip install black
pip install py-solc-x
solcx.install_solc('0.6.0')

In [40]:
from solcx import compile_standard
import json
from web3 import Web3
import os
from dotenv import load_dotenv

#Load Env Variables
load_dotenv()
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
ADDRESS = os.getenv("ADDRESS")

compiled_sol = compile_standard(
    {
    "language": "Solidity",
    "sources": {"SimpleStorage.sol" : {"content": simple_storage_file}},
    "settings": {
        "outputSelection" : {
            "*" : {
                "*" : ["abi","metadata","evm.bytecode","evm.sourceMap"]}
        }
    },
    },
    solc_version = "0.6.0"
    )

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

#get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
#get abi
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

#connecting to ganache
w3 = Web3(Web3.HTTPProvider("https://rinkeby.infura.io/v3/2e8ad46c0cb241aeb14d4e5f341db92b"))
chain_id = 4 #Rinkeby is 4

#Create Contract in Python
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
print(SimpleStorage)

#Get the latest transaction
nonce = w3.eth.getTransactionCount(ADDRESS)
#Build Transaction
transaction = SimpleStorage.constructor().buildTransaction({"chainId":chain_id,"gasPrice": w3.eth.gas_price,"from":ADDRESS, "nonce": nonce})
#Sign it
signed_txn = w3.eth.account.sign_transaction(transaction,private_key=PRIVATE_KEY)
#Send it
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction) 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

<class 'web3._utils.datatypes.Contract'>


In [41]:


#Working with the Contract
##Contract Address -->from the receipt
##Contract ABI --> from the compiled sol jsonA ABI extract
simple_storage = w3.eth.contract(address=tx_receipt.contractAddress,abi=abi)
# Call ->Simulate making the call and getting a return value (does not make a change in value (blue buttons in remix))
# Transact --> Orange butons in remix. They do state changes. 
store_transaction = simple_storage.functions.store(15).buildTransaction({"chainId":chain_id,"gasPrice": w3.eth.gas_price,"from":ADDRESS, "nonce": nonce + 1})
#Sign it
signed_store_txn = w3.eth.account.sign_transaction(store_transaction,private_key=PRIVATE_KEY)
#Send it
send_store_tx = w3.eth.send_raw_transaction(signed_store_txn.rawTransaction) 
tx_receipt = w3.eth.wait_for_transaction_receipt(send_store_tx)
print(simple_storage.functions.retrieve().call())

15
